In [1]:
import cv2
import numpy as np
from enum import Enum

WEIGHTS = 'weights/2019-10-07_03-25-35_epoch 35_r2 0.9168.h5'

DEVICE = 0
WIDTH = 224
HEIGHT = 224

class State(Enum):
    SEARCH = 1
    TRACK = 2

"""
User Interface Stuff (You can ignore this)
"""
import IPython
import ipywidgets as widgets
from IPython.display import display

ipython = IPython.get_ipython()

style = {'description_width': 'initial'}

w_image = widgets.Image(width=WIDTH, height=HEIGHT, format='png',
                        layout=widgets.Layout(width='100%'))
w_thresh = widgets.FloatSlider(min=0, max=1, value=0.5, step=0.01, 
                                             description='Minimum Tracking Confidence',
                                             layout=widgets.Layout(width='100%'),
                                             style=style)

vbox = widgets.VBox([w_image, w_thresh])

display(vbox)

[-24.91812134 -10.66537476  -1.25799561  -8.83425903]
[-1.26991272 -1.0227356   5.48812866 11.48181152]
[ 14.72883606  -3.59763336 -15.75860596  12.1842041 ]
[-7.66442871 -4.74354553  6.05224609  9.49963379]
[-2.6371155 -0.6914673 12.943298  16.378296 ]
[ 9.41726685  6.08816528 16.54205322 27.70825195]
[-14.79008484   0.75416565  16.20663452   8.87435913]
[-15.35965   -8.398773   5.5961     2.673706]
[-13.889679   -6.5677032  -1.1573486  -3.4169006]
[-3.0980682 -3.8220673  7.5003357 17.020935 ]
[ 4.3677673 -4.652313   2.8465576 21.939636 ]
[-9.745026  -0.7854614  7.412628   8.033264 ]
[-4.732422  -9.211792  -4.935791   6.2509766]
[-11.52475    -5.6148224   7.7250977  11.443665 ]
[ -6.0536194 -12.25528     2.8154907  18.31659  ]
[-42.04138     2.5952454  51.063416   16.360657 ]
[-18.577332    -0.20985413  -6.3494263  -19.256958  ]
[ 16.215149   -3.7911682 -35.76129   -11.067932 ]
[-0.877594  -3.2397156 -9.913086  -4.3372803]
[ 28.704956   -1.5482178 -28.854004    6.168701 ]
[-14.977905 

[-0.9448242   0.46647644 -2.7328186  -3.2539978 ]
[ 23.544662    2.1117706 -11.390198   13.844757 ]
[-18.444107   -3.322647    3.4432068 -10.194824 ]
[-1.037857  -2.405136  -3.512329   1.0223694]
[-3.4792175   3.0568237   8.224182    0.30444336]
[ 2.797699  -1.3578796 -3.2659302  4.671692 ]
[ -0.313385   -6.365509  -16.940186   -5.9926147]
[-18.249146    2.8677216  20.591217    0.5111084]
[ 14.706436    -0.12423706 -15.928406     2.0509644 ]
[ 6.9957733e+00  2.6151581e+00 -1.6174316e-03  9.9042053e+00]
[-14.982681    2.4512482  18.500397    3.1384888]
[ -8.483398    1.0316162  -2.7937317 -10.877319 ]
[ 11.831329  -5.863968 -15.336914   9.272949]
[-3.5259857  3.70253   10.221985   5.7901917]
[ -8.889481    3.094986   -1.3963318 -10.025085 ]
[ 2.085434   2.3148499 -3.9959106 -1.0219116]
[  9.630386    -0.33685303 -13.954071     0.9163208 ]
[2.847931  3.9537964 2.5904846 6.406769 ]
[-14.814102   -1.8627167   1.7540894  -5.2933655]
[ -3.2131195  -8.431717  -22.421448  -11.204346 ]
[-10.297

[ 9.49751282  2.57608032  7.51260376 17.73141479]
[ 22.04983521  -2.03955078 -12.97238159  11.08755493]
[-14.40917969  -9.94511414   2.13684082  -3.36022949]
[-44.74066162  -4.37413025  57.95272827  17.98348999]
[-34.6597      -9.760696    23.052338    -0.63394165]
[-15.08004761  -2.51377869  19.79891968   9.70480347]
[-6.54492188  0.09025574 -7.1211853  -8.59268188]
[  9.85437012  -3.24388123 -24.05183411  -5.52667236]
[ 18.42034912   1.48114014 -22.3709259    1.99060059]
[ 16.65762329   1.60427856 -21.76377869   2.92166138]
[-25.44332886   1.99772644  19.92572021  -1.46200562]
[  1.4948273   0.5852661 -16.617615   -9.926605 ]
[-2.5446777   0.75379944 -8.511414   -2.6784363 ]
[  3.994461   -6.8967743 -23.134308   -4.670532 ]
[ 5.7217865 -0.6469879  6.938263  23.896393 ]
[-24.793152    0.2066803  25.874756    4.6710815]
[-5.5711975   1.3430481   0.18469238 -3.0300293 ]
[ 1.4849854 -2.0994263 -9.088379  -0.8975525]
[-0.60173035 -5.2983856  -8.987701   -0.73464966]
[-2.5250092 -1.9640503

[ 19.723541    -0.49996948 -12.632935    17.447876  ]
[-14.802368   -3.5674438  30.39383    29.729492 ]
[-24.896103     0.62472534  34.535767    13.700806  ]
[16.81247    8.9661255  9.807251  15.798889 ]
[ 48.17974854 -46.11401367 -58.02514648  44.50769043]
[-16.190979    5.1540985  40.65381    27.89148  ]
[-14.99653625  -7.68916321  13.20849609  19.10144043]
[ 25.64419556  -0.24169922 -32.43728638   2.57992554]
[ 28.73078918   0.54840088 -31.41162109   1.87698364]
[ 28.26791382   1.06512451 -30.73846436   4.68817139]
[ 27.97038269  -4.51887512 -44.59906006  -5.09832764]
[ 28.12931824  -3.13280487 -45.02905273  -5.70220947]
[ 27.81494141  -2.9709549  -41.76791382  -2.69717407]
[ 30.85275269  -3.64034271 -41.45855713   0.47711182]
[ 29.49212646  -3.04481506 -40.04055786  -0.42123413]
[ 21.97097778  -1.10308075 -35.80767822  -3.88461304]
[ 19.75245667  -5.04774475 -36.06915283  -2.59988403]
[ 24.36442566  -4.04095459 -38.27197266  -1.28866577]
[ 19.52215576  -2.37161255 -24.26037598   6.

[ 3.6386108  1.0414734 -6.330353  -6.911133 ]
[ 7.679199  -0.9158783 -3.769287   7.4329834]
[-1.3755035  1.9919128  3.3822327 -2.0847778]
[-1.5959167 -3.0533905 -9.764862  -8.594482 ]
[ -1.1386871  -4.2981567 -10.00296    -8.627869 ]
[-6.6391907  3.3839264  2.3742676 -3.878601 ]
[ -8.50885     1.0478058  -9.263702  -14.78833  ]
[ 68.90606689 -90.73178101 -38.80426025 133.19378662]
[-7.502014 16.22406  14.235352 -6.336914]
[ 13.403534  12.000519 -20.202026 -15.84082 ]
[ 23.17025757   3.05625916 -19.57055664   7.12976074]
[-26.2104187  -11.008255    15.61184692   4.94152832]
[-11.64828491  -8.06733704  19.9515686   23.46994019]
[-21.02168274  -7.0632782   21.16555786  11.38406372]
[-7.15751648 -1.90704346 24.51226807 23.2322998 ]
[-8.75221252 -5.47686768  7.13504028  6.36785889]
[ 24.85891724   1.81536865 -23.14309692   9.43695068]
[ 21.0933075    2.3442688  -22.55493164   5.99105835]
[-8.10954285 -3.70770264  9.10409546 11.76748657]
[-13.247589   -5.5794067   5.4476013   3.3273926]
[-37

In [2]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True 
sess = tf.Session(config=config)
set_session(sess) 

from model import build_model

model, _, _ = build_model(weights=None)
model.load_weights(WEIGHTS, by_name=True)

Using TensorFlow backend.
W1006 22:37:39.412586 140445905016640 deprecation_wrapper.py:119] From /home/carroll/anaconda3/envs/ryzen/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1006 22:37:39.415573 140445905016640 deprecation_wrapper.py:119] From /home/carroll/anaconda3/envs/ryzen/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1006 22:37:39.425517 140445905016640 deprecation_wrapper.py:119] From /home/carroll/anaconda3/envs/ryzen/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1006 22:37:39.425985 140445905016640 deprecation_wrapper.py:119] From /home/carroll/anaconda3/envs/ryzen/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name t

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_left (InputLayer)         (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_right (InputLayer)        (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_left[0][0]                 
                                                                 input_right[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 8)  216         conv1_pad[0][0]                  
          

In [3]:
cap = cv2.VideoCapture(4)

state = State.SEARCH
objects = []
img_initial = None
last_pos = None

try:
    while True:
        # Have to call this to get update values from sliders / dropdowns
        ipython.kernel.do_one_iteration()
        
        # Read the frame from the camera
        ret, frame = cap.read()
        
        if frame.shape[0] < frame.shape[1]:
            frame = cv2.copyMakeBorder(frame, 0, frame.shape[1] - frame.shape[0], 0, 0, cv2.BORDER_CONSTANT, 0)
        elif frame.shape[0] > frame.shape[1]:
            frame = cv2.copyMakeBorder(frame, 0, 0, 0, frame.shape[0] - frame.shape[1], cv2.BORDER_CONSTANT, 0)
            
        frame_search = frame

        img_draw = frame_search.copy()
        
        if state == State.SEARCH:
                    
            # Resize the frame to a lower resolution for performance reasons
            img_original = cv2.resize(frame, (WIDTH, HEIGHT))
            search_ratio = WIDTH/frame.shape[0]

            # Convert image to greyscale
            img_grey = cv2.cvtColor(img_original, cv2.COLOR_BGR2GRAY)
            
            # Apply a slight guassian blur (low pass filter) to filter out noise
            img_blur = cv2.GaussianBlur(img_grey, (3, 3), 0)
    
            img_final = img_blur
            
            # Only compute the difference image if we have both a current and previous frame
            if img_initial is not None:
                
                # By looking at the difference between our two images, we can find motion
                img_diff = cv2.absdiff(img_initial, img_final)
    
                # Quantizes pixels to only represent ON or OFF instead of a wide range of intensity values
                _, img_threshold = cv2.threshold(img_diff, 32, 255, cv2.THRESH_BINARY)
    
                # Each ON pixel expands in every direction.
                img_dilate = cv2.dilate(img_threshold, None, iterations=8)
                
                # A contour is a group of "on" pixels that are connected to eachother
                # Here we represent contours by the lines that form their edges
                cnts = cv2.findContours(img_dilate, cv2.RETR_EXTERNAL,
                                       cv2.CHAIN_APPROX_SIMPLE)[0]
    
                
                # Iterate through every contour we found and determine if it should get a bounding box
                for c in cnts:
                    # If the contour is too small, ignore it
                    if cv2.contourArea(c) < int(0.026 * WIDTH * HEIGHT):
                        continue
    
                    # Convert the contour to a rectangle starting at (x, y) with a width w and height h
                    (x, y, w, h) = cv2.boundingRect(c)
                                
                    x, y, w, h = np.array([x, y, w, h])/search_ratio
                    x, y, w, h = int(x), int(y), int(w), int(h)
                    
                    cv2.rectangle(img_draw, (x, y), (x + w, y + h), (0, 0, 0), 2)
                    
                    objects.append([x, y, x+w, y+h])
                    
                    state = State.TRACK
                    img_initial = None
                
            # Visualization Code
            result, img_png = cv2.imencode('.png', img_draw)
            w_image.value = img_png.tobytes()

            img_initial = img_final
            
        elif state == State.TRACK:                        
            prune = []
                        
            for idx, obj in enumerate(objects):
                
                x1, y1, x2, y2 = obj
                
                siamese_images = np.zeros((2, 224, 224, 3), dtype=np.float32)

                width = x2 - x1
                height = y2 - y1

                center = np.round((np.array([x1, y1]) + np.array([x2, y2])) / 2)
                size = 2*max(width, height)

                size_half = np.ceil(size / 2)

                sx1 = center[0] - size_half
                sy1 = center[1] - size_half

                sx2 = center[0] + size_half
                sy2 = center[1] + size_half

                # Caclulate padding
                s_pad_left = int(abs(sx1)) if sx1 < 0 else 0
                s_pad_right = int(sx2 - frame.shape[1]) if sx2 > frame.shape[1] else 0

                s_pad_top = int(abs(sy1)) if sy1 < 0 else 0
                s_pad_bot = int(sy2 - frame.shape[0]) if sy2 > frame.shape[0] else 0

                pad_bot = s_pad_bot + 100
                pad_top = s_pad_top + 100

                pad_left = s_pad_left + 100
                pad_right = s_pad_right + 100

                # Recalculate crop regions after padding
                x1 += pad_left
                x2 += pad_left

                y1 += pad_top
                y2 += pad_top

                center = np.round((np.array([x1, y1]) + np.array([x2, y2])) / 2)

                sx1 = center[0] - size_half
                sy1 = center[1] - size_half

                sx2 = center[0] + size_half
                sy2 = center[1] + size_half

                # Pad the actual images
                image_padded_track = cv2.copyMakeBorder(frame_track,
                                                  pad_top, pad_bot, pad_left, pad_right,
                                                  cv2.BORDER_CONSTANT, value=0)
                image_cropped_track = image_padded_track[int(sy1):int(sy2), int(sx1):int(sx2)]
                image_resized_track = cv2.resize(image_cropped_track, (224, 224), interpolation=cv2.INTER_LINEAR)
                siamese_images[0] = image_resized_track

                image_padded_search = cv2.copyMakeBorder(frame_search,
                                                  pad_top, pad_bot, pad_left, pad_right,
                                                  cv2.BORDER_CONSTANT, value=0)
                image_cropped_search = image_padded_search[int(sy1):int(sy2), int(sx1):int(sx2)]
                image_resized_search = cv2.resize(image_cropped_search, (224, 224), interpolation=cv2.INTER_LINEAR)
                siamese_images[1] = image_resized_search
                
                track_ratio = 224/(sy2 - sy1)

                # Jointly Normalize both images
                siamese_images -= np.mean(siamese_images, axis=(0, 1, 2))
                siamese_images /= np.std(siamese_images, axis=(0, 1, 2)) + np.finfo(np.float32).eps

                regr, cls = model.predict([[siamese_images[0]], [siamese_images[1]]])
                
                regr = regr[0]
                cls = cls[0]
                
                x1, y1, x2, y2 = regr
                
                regr = regr/track_ratio
                regr += [sx1-pad_left, sy1-pad_top, sx1-pad_left, sy1-pad_top]
                
                x1, y1, x2, y2 = regr
                
                print(np.array(regr) - np.array(objects[idx]))
                
                objects[idx] = regr
                
                if cls < w_thresh.value:
                    prune.append(idx)
                    cv2.rectangle(img_draw, (int(x1), int(y1)), (int(x2), int(y2)) , (0, 0, 255), 2)
                else:
                    pass
                    cv2.rectangle(img_draw, (int(x1), int(y1)), (int(x2), int(y2)) , (0, 255, 0), 2)

                result, img_png = cv2.imencode('.png', img_draw)
                w_image.value = img_png.tobytes()
                    
            prune.reverse()
            for p in prune:
                del objects[p]
                
            if len(objects) == 0:
                state = State.SEARCH



        frame_track = frame.copy()

        
except KeyboardInterrupt:
    pass
finally:
    cap.release()


[ 62.86587524 -45.72616577 -55.54553223  76.71948242]
[ 70.06845093 -37.36520386 -18.69616699  98.15753174]
[-4.26983643  2.87097168  7.72460938  1.3086853 ]
[11.60183716 -6.18133545 -5.60986328 13.33886719]
[ 7.16183472 -3.40823364 -8.4666748   6.3286438 ]
[ 6.87683105 -6.25662231  4.44723511 19.17044067]
[ -4.41238403  -9.68983459 -22.04040527 -15.96118164]
[ 4.12666321  1.17553711 12.98513794 15.30706787]
[ 33.21676636  -3.30708313 -32.54125977   4.3460083 ]
[-11.15890503  -3.92071533  -1.65466309  -3.05621338]
[1.64085388 6.36834717 9.28201294 2.54458618]
[ -5.03051758 -71.30758667 -28.97088623  34.19506836]
[ 21.17388916  -3.72465515 -22.81915283   6.89978027]
[  4.28710938 -33.03820801   9.04840088  44.61779785]
[26.62445068 -5.90424347 -9.79302979 29.55950928]
[ 4.17956543  3.275177   11.04064941 14.33538818]
[ 5.60214233  0.01327515  6.92401123 12.69454956]
[10.02393341 -0.45974731 -7.31661987  0.76904297]
[ 12.73401642  -9.84106445 -23.28024292  -2.36755371]
[ 30.26068115 -10.